# Exploratory Data Analysis and Experimentation for LitQA2 Benchmark

In [ ]:
# Import libraries
import pandas as pd
from paperqa import ask, Settings

In [ ]:
# Import the data from LitQA2 data from Hugging Face

litqa2 = 